# Pré-Requisitos

## PROV - https://pypi.org/project/prov/

In [ ]:
!pip install prov

## Sweetviz - https://pypi.org/project/sweetviz/

In [ ]:
!pip install sweetviz

# Bibliotecas

In [ ]:
import sweetviz as sv
import pandas as pd
import prov
import json

# Reprodutibilidade

## Mostra as versões das bibliotecas usadas

In [ ]:
import sys

print('Versão do Pandas:', pd.__version__)
print('Versão do PROV:', prov.__version__)
print('Versão do JSON:', json.__version__)
print('Versão do Sweetviz:', sv.__version__)
print('Versão do Python:', sys.version)

## Sistema Operacional

In [ ]:
!cat /etc/os-release

### Versão do kernel

In [ ]:
!cat /proc/version

### Arquitetura do Processador (32 ou 64 bits)

In [ ]:
!lscpu

### Memória (RAM e afins)

In [ ]:
!cat /proc/meminfo

# Provenance

In [ ]:
from prov import model
# Create a new provenance document
doc = model.ProvDocument()
# Declaring namespaces
doc.set_default_namespace('https://github.com/FundamentosDataScienceEleicoesRJ2020')
doc.add_namespace('cores', 'http://labcores.ppgi.ufrj.br/')
doc.add_namespace('ppgi', 'http://www.ppgi.ufrj.br/')
# Criando entidades
dataset = doc.entity('cores:prefeito_rio_2020', {model.PROV_TYPE : 'cores:dataset', model.PROV_LOCATION : 'https://rafaelescalfoni.net/prefeito_rio_2020.csv',
                                                 'cores:resume' : 'Dados coletados do twitter sobre tweets que usaram em seu texto termos relacionados aos candidatos à prefeitura do Rio de Janeiro durante as eleições de 2020',
                                                 'cores:records' : '580.354', 'cores:columns' : '31'})
csv = doc.entity('cores:prefeito_rio_2020.csv', {model.PROV_TYPE : 'File', 'cores:size' : '1.29 GB'})
clean_data = doc.entity('ppgi:Dataset com Dados Tratados')
proc_data = doc.entity('ppgi:Dataset com Texto Processado')
artigo = doc.entity('ppgi:Artigo sobre Análise Exploratória de Dados - Eleições de 2020 para a Prefeitura do Rio de Janeiro')
# Criando agentes
jamile = doc.agent('cores:Jamile Santos', {model.PROV_TYPE : 'prov:Person', 'cores:position': 'Estudante',
                                           model.PROV_LOCATION : 'Lab CORES - Laboratório de Computação Social e Análise de Redes Sociais'})
jivago = doc.agent('cores:Jivago Medeiros', {model.PROV_TYPE : 'prov:Person', 'cores:position': 'Estudante',
                                             model.PROV_LOCATION : 'Lab CORES - Laboratório de Computação Social e Análise de Redes Sociais'})
api = doc.agent('Twitter API', {model.PROV_TYPE : 'prov:SoftwareAgent', 'prov:publisher' : 'Twitter', model.PROV_LOCATION : 'https://api.twitter.com/2/tweets/search/stream',
                                'prov:label' : 'https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/api-reference/get-tweets-search-stream'})
grupo = doc.agent('ppgi:Grupo 1', {model.PROV_TYPE : 'prov:Organization', model.PROV_ROLE : 'Grupo de Trabalho', model.PROV_LOCATION : 'Fundamentos de Ciência de Dados / PPGI-UFRJ',
                                   'ppgi:members' : 'Ana Vasconcelos, Jivago Medeiros, Rafael Escalfoni, Silas Pereira, Sírius Thadeu'})
# Criando atividades
scrap = doc.activity('cores:Scrap Text and Metadata', '29/09/2020', '01/12/2020', {'cores:filter' : '[{nomeCandidato} AND [{numeroPartido} OR {siglaPartido} OR $ANO]]'})
data_clean = doc.activity('ppgi:Limpeza de Dados')
text_proc = doc.activity('ppgi:Processamento de Texto')
eda = doc.activity('ppgi:Análise Exploratória de Dados')
# Relacionamentos
doc.used(data_clean, dataset)
doc.used(text_proc, clean_data)
doc.used(eda, proc_data)
doc.wasInfluencedBy(scrap, api)
scrap.wasAssociatedWith(jamile)
scrap.wasAssociatedWith(jivago)
data_clean.wasAssociatedWith(grupo)
text_proc.wasAssociatedWith(grupo)
eda.wasAssociatedWith(grupo)
dataset.wasAttributedTo(jamile)
dataset.wasAttributedTo(jivago)
dataset.wasGeneratedBy(scrap)
csv.wasDerivedFrom(dataset)
clean_data.wasGeneratedBy(data_clean)
clean_data.wasDerivedFrom(csv)
proc_data.wasGeneratedBy(text_proc)
proc_data.wasDerivedFrom(clean_data)
artigo.wasGeneratedBy(eda)
artigo.wasDerivedFrom(proc_data)
artigo.wasAttributedTo(grupo)
print(doc.get_provn())

## Gera o gráfico

In [ ]:
from prov import dot
# Save the provenance graph
dot = dot.prov_to_dot(doc)
dot.write_png('prefeito_rio_2020-dataset-prov.png')
# Visualize the graph file
from IPython.display import Image
Image('prefeito_rio_2020-dataset-prov.png')